In [ ]:
import numpy as np
import scipy.linalg as la
import pickle

from scipy.optimize import minimize
from itertools import product

from qiskit import *
from qiskit.quantum_info.states.random import random_density_matrix
from qiskit.quantum_info import Statevector, DensityMatrix, partial_trace

import matplotlib.pyplot as plt
from mpl_axes_aligner import shift
plt.rcParams.update(
    {"text.usetex": True, "font.family": "serif", "font.size": 10}
)

In [ ]:
def ansatz_v1(n, theta):
    circ = QuantumCircuit(n)
    for tno, t in enumerate(theta):
        circ.rz(t[0], [0])
        circ.ry(t[1], [0])
        circ.rz(t[2], [0])
        circ.rz(t[3], [1])
        circ.ry(t[4], [1])
        circ.rz(t[5], [1])
        
        circ.cx([0], [1])
        
        circ.rz(t[6], [0])
        circ.ry(t[7], [0])
        circ.rz(t[8], [0])
        circ.rz(t[9], [1])
        circ.ry(t[10], [1])
        circ.rz(t[11], [1])
    return circ

def ansatz_v2(n, theta):
    circ = QuantumCircuit(n)
    for tno, t in enumerate(theta):
        circ.rz(t[0], [0])
        circ.ry(t[1], [0])
        circ.rz(t[2], [0])
        circ.rz(t[0], [1])
        circ.ry(t[1], [1])
        circ.rz(t[2], [1])
        
        circ.cx([0], [1])
        
        circ.rz(t[0], [0])
        circ.ry(t[1], [0])
        circ.rz(t[2], [0])
        circ.rz(t[0], [1])
        circ.ry(t[1], [1])
        circ.rz(t[2], [1])
    return circ

def ansatz_v3(n, theta):
    circ = QuantumCircuit(n)
    circ.ry(theta[0], [0])
    circ.rz(theta[1], [0])
    circ.ry(theta[0], [1])
    circ.rz(theta[1], [1])
    circ.cx(0,1)
    return circ

def ansatz_v4(n, theta):
    circ = QuantumCircuit(n)
    for tno, t in enumerate(theta):
        circ.rz(t[0], [0])
        circ.ry(t[1], [0])
        circ.rz(t[2], [1])
        circ.ry(t[3], [1])
        
        circ.cx([0], [1])
    return circ

def cost_func(n, theta, dm, trace_before_diag):
    """returns the cost function value"""
    """for particular choice of layered ansatz and angle"""
    theta = theta.reshape(-1, 12)
    dm_evo = dm.evolve(ansatz_v1(n, theta))
    dm_evo = np.kron(dm_evo.data, dm_evo.data)
    
    qc_cost = QuantumCircuit(2*n)
    for i in range(n):
        qc_cost.cx([i+n], [i])
    dm_evo = DensityMatrix(dm_evo).evolve(qc_cost)
    
    par_trace = partial_trace(dm_evo, range(n))
    trace_after_diag = np.trace(par_trace.data @ par_trace.data).real    
    return trace_before_diag - trace_after_diag

def cost_func_v2(n, theta, dm, trace_before_diag):
    """returns the cost function value"""
    """for particular choice of layered ansatz and angle"""
    dm = np.kron(dm.data, dm.data)
    theta = theta.reshape(-1, 3)
    qc_cost = QuantumCircuit(2*n)
    qc_cost = qc_cost.compose(ansatz_v4(n, theta), range(n))
    qc_cost = qc_cost.compose(ansatz_v4(n, theta), range(n, 2*n))
    for i in range(n):
        qc_cost.cx([i+n], [i])
#     print(qc_cost)
    dm_evo = DensityMatrix(dm).evolve(qc_cost)
    par_trace = partial_trace(dm_evo, range(n))
    trace_after_diag = np.trace(par_trace.data @ par_trace.data).real    
    return trace_before_diag - trace_after_diag

def eigenvals_func(dm, ansatz):
    """Return the eigenvalues"""
#     qc_eigenvals = ansatz
    dm_evo = DensityMatrix(dm).evolve(ansatz)
    dm_evo = dm_evo.data
    diagnls = list((np.diagonal(dm_evo)).real)
    diag_A = []
    diag_A.append(diagnls)
    return diag_A, dm_evo

def cost_func_landscape(n, ang1, ang2, dm, trace_before_diag):
    """returns the cost function value"""
    """for particular choice of layered ansatz and angle"""
#     print(ang1, ang2)
    theta = [ang1, ang2]
    dm_evo = dm.evolve(ansatz_v3(n, theta))
    dm_evo = np.kron(dm_evo.data, dm_evo.data)
    
    qc_cost = QuantumCircuit(2*n)
    for i in range(n):
        qc_cost.cx([i+n], [i])
    dm_evo = DensityMatrix(dm_evo).evolve(qc_cost)
    
    par_trace = partial_trace(dm_evo, range(n))
    trace_after_diag = np.trace(par_trace.data @ par_trace.data).real    
    return trace_before_diag - trace_after_diag

In [ ]:
qubits, rank, total_seed = 2,4,50
for seed in range(1, total_seed+1):
    rdm = random_density_matrix(2**qubits, rank = rank, seed = seed)
    rdm_data = rdm.data
    trace_before_diag = np.trace(np.matmul(rdm_data,rdm_data)).real
    true_eig = np.sort(la.eig(rdm_data)[0].real)
    print(true_eig)
    def cost(theta):
        return cost_func(qubits, theta, rdm , trace_before_diag)
    layers = 6
#     layers = 6
    hea_data = {}
    layer_list, eig_err_list = [], []
    for l in range(1, layers+1):
        print('layer', l)
        cost_list, ang_list = [], []
        for _ in range(10):
            initial_guess = np.random.random(12*l)
            res = minimize(cost, initial_guess, method = 'COBYLA', options={'maxiter': 400})
            cost_list.append(res['fun'])
            ang_list.append(res['x'])
        
        theta = ang_list[cost_list.index(np.min(cost_list))]
        theta = theta.reshape(-1,12)
        ansatz = ansatz_v1(qubits, theta)
        inf_eig = np.sort(eigenvals_func(rdm, ansatz)[0])[0]
        
        err = 0
        for eigno in range(2**qubits):
            err += np.abs(inf_eig[eigno] - true_eig[eigno])**2
        print(err)
        layer_list.append(l)
        eig_err_list.append(err)
    hea_data['layer_list'] = layer_list
    hea_data['eig_err_list'] = eig_err_list
    with open(f'plot_data/{qubits}_qubit_mixed_rank_{rank}_seed_{seed}_lHEA_data_tot_state_{total_seed}.p', 'wb') as fp:
        pickle.dump(hea_data, fp)
    print('-----------')

In [ ]:
# COST FUNCTION LANDSCAPE

In [ ]:
qubits, seed, rank = 2, 1, 2
rdm = random_density_matrix(2**qubits, rank = rank, seed = seed)
rdm_data = rdm.data
trace_before_diag = np.trace(np.matmul(rdm_data,rdm_data)).real
true_eig = np.sort(la.eig(rdm_data)[0].real)
ang_list1, ang_list2 = np.linspace(-2*np.pi, 2*np.pi, 50), np.linspace(-2*np.pi, 2*np.pi, 50)
X,Y = np.meshgrid(ang_list1, ang_list2)


def sum(X, Y):
    return X+Y

Z = sum(X, Y).T
Z = []
for x in ang_list1:
    z2 = []
    for y in ang_list2:
        cost = cost_func_landscape(qubits, x, y, rdm, trace_before_diag)
        z2.append(cost)
    Z.append(z2)
Z = np.asarray(Z)

In [ ]:
from mpl_toolkits.mplot3d.axes3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib

fig = plt.figure(figsize=(6,4))

# `ax` is a 3D-aware axis instance because of the projection='3d' keyword argument to add_subplot
# ax = fig.add_subplot(1, 1, 1, projection='3d')

# p = ax.plot_surface(X, Y, Z, rstride=4, cstride=4, linewidth=0)

# surface_plot with color grading and color bar
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.set_xlabel('$\\theta_1$')
ax.xaxis.set_major_locator(plt.MultipleLocator(np.pi ))
ax.xaxis.set_minor_locator(plt.MultipleLocator(np.pi / 12))
ax.xaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter()))
ax.set_ylabel('$\\theta_2$')
ax.yaxis.set_major_locator(plt.MultipleLocator(np.pi ))
ax.yaxis.set_minor_locator(plt.MultipleLocator(np.pi / 12))
ax.yaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter()))
ax.set_zlabel('$C(\\theta)$')
p = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=matplotlib.cm.coolwarm, linewidth=0, antialiased=False)
fig.tight_layout()
# fig.savefig('plot/2_qubit_random_state_cost_landscape.pdf')
# fig.savefig('plot/2_qubit_random_state_cost_landscape.png')
# cb = fig.colorbar(p, shrink=0.5)

In [ ]:
def low_rank_approx(SVD=None, A=None, r=1):
    if not SVD:
        SVD = np.linalg.svd(A, full_matrices=False)
    u, s, v = SVD
    Ar = np.zeros((len(u), len(v)))
    for i in range(r):
        Ar = np.add(Ar, s[i] * np.outer(u.T[i], v[i]))
    return Ar

In [ ]:
qubits, rank, seed = 4,4,10

state_data = []
with (open(f"state_data/{qubits}_qubit_reduce_heisen_model_state_data.p", "rb")) as openfile:
    while True:
        try:
            state_data.append(pickle.load(openfile))
        except EOFError:
            break
state = state_data[0]['state']

In [ ]:
u, s, v = np.linalg.svd(state.data)
low_rank_state = low_rank_approx((u,s,v), np.asmatrix(state.data), r=6)

In [ ]:
la.eig(state.data)[0].real

In [ ]:
la.eig(low_rank_state.data)[0].real

In [ ]:
import time
qubits, rank, seed = 4,4,10

state_data = []
with (open(f"state_data/{qubits}_qubit_reduce_heisen_model_state_data.p", "rb")) as openfile:
    while True:
        try:
            state_data.append(pickle.load(openfile))
        except EOFError:
            break

rdm = state_data[0]['state']
rdm_data = rdm.data
rdm = random_density_matrix(2**qubits, rank = rank, seed = seed)
rdm_data = rdm.data
dm = np.kron(rdm_data,rdm_data)
trace_before_diag = np.trace(np.matmul(rdm_data,rdm_data)).real
true_eig = np.sort(la.eig(rdm_data)[0].real)
time1 = time.time()
def cost(theta):
    return cost_func(qubits, theta, dm, trace_before_diag)
layers = 6
hea_data = {}
layer_list, eig_err_list = [], []
for l in range(1, layers+1):
    print('layer', l)
    cost_list, ang_list = [], []
    for _ in range(5):
        initial_guess = np.random.random(2*qubits*l)
        res = minimize(cost, initial_guess, method = 'COBYLA',options={'maxiter': 500})
        cost_list.append(res['fun'])
        ang_list.append(res['x'])
    time2 = time.time()
    print(time2-time1)
    theta = ang_list[cost_list.index(np.min(cost_list))]
    theta = theta.reshape(-1,2*qubits)
    inf_eig = np.sort(eigenvals_func(theta, rdm, qubits)[0])[0]
    err = 0
    for eigno in range(qubits**2):
        err += np.abs(inf_eig[eigno] - true_eig[eigno])**2
    print(err)

In [ ]:
theta = np.random.random(12)
theta = theta.reshape(-1,6)
print(theta)
print(ansatz_v1(2, theta))

In [ ]:
def def_action_list(qubit):
    if qubit == 2:
        action_list = [ [0,1, 0,2],
                        [1,0, 0,2],
                        [2,0, 0,0],
                        [2,0, 1,0],
                        [2,0, 2,0],
                        [2,0, 0,1],
                        [2,0, 1,1],
                        [2,0, 2,1]]
    elif qubit == 3:
        action_list = [ [0,1, 0,3],
                        [1,0, 0,3],
                        [2,0, 0,3],
                        [0,2, 0,3],
                        [1,2, 0,3],
                        [2,1, 0,3],
                        [3,0, 0,0],
                        [3,0, 1,0],
                        [3,0, 2,0],
                        [3,0, 0,1],
                        [3,0, 1,1],
                        [3,0, 2,1],
                        [3,0, 0,2],
                        [3,0, 1,2],
                        [3,0, 2,2]]
    return action_list

In [ ]:
# STEP WISE OPTIMIZATION

In [ ]:
def init_quantum_circ(qubit, action_after_opt, ang_after_opt):
    init_circ = QuantumCircuit(qubit)
    if len(action_after_opt) != 0:
        ctrl, targ = action_after_opt[0], action_after_opt[1]
        rot_axis, rot_qubit = action_after_opt[2], action_after_opt[3]
        if ctrl != qubit:
            init_circ.cx(ctrl, targ)
        if rot_qubit != qubit:        
            if rot_axis == 0:
                init_circ.rx(ang_after_opt[0], rot_qubit)
            elif rot_axis == 1:
                init_circ.ry(ang_after_opt[0], rot_qubit)
            elif rot_axis == 2:
                init_circ.rz(ang_after_opt[0], rot_qubit)
    return init_circ
                

def ansatz(qubit, action, action_after_opt, ang_after_opt, theta):
    ansatz = init_quantum_circ(qubit, action_after_opt, ang_after_opt)
#     print(ansatz)
    ctrl, targ = action[0], action[1]
    rot_axis, rot_qubit = action[2], action[3]
    if ctrl != qubit:
        ansatz.cx(ctrl, targ)
    elif rot_qubit != qubit:
#         print(action)
        if rot_axis == 0:
            ansatz.rx(0, rot_qubit)
        elif rot_axis == 1:
            ansatz.ry(0, rot_qubit)
        elif rot_axis == 2:
            ansatz.rz(0, rot_qubit)
    return ansatz

def cost_func(n, theta, action, action_after_opt, ang_after_opt, dm, trace_before_diag):
    """returns the cost function value"""
    """for particular choice of layered ansatz and angle"""
    ansatz_now = ansatz(n, action, action_after_opt, ang_after_opt, theta)
    dm_evo = dm.evolve(ansatz_now)
    dm_evo = np.kron(dm_evo.data, dm_evo.data)
    qc_cost = QuantumCircuit(2*n)
    for i in range(n):
        qc_cost.cx([i+n], [i])
    dm_evo = DensityMatrix(dm_evo).evolve(qc_cost)
    
    par_trace = partial_trace(dm_evo, range(n))
    trace_after_diag = np.trace(par_trace.data @ par_trace.data).real    
    return trace_before_diag - trace_after_diag

def optimal_action(cost_action_dict):
    cost_list = []
    for key in cost_action_dict.keys():
        cost_list.append(cost_action_dict[key])
        key_list.append(key)
    indx = cost_list.index(np.min(cost_list))
    return def_action_list(qubit=qubits)[indx], indx

In [ ]:
qubits, rank, seed = 2,4,1
rdm = random_density_matrix(2**qubits, rank = rank, seed = seed)
rdm_data = rdm.data
trace_before_diag = np.trace(np.matmul(rdm_data,rdm_data)).real
true_eig = np.sort(la.eig(rdm_data)[0].real)

action_after_opt, ang_after_opt = [], [0]
for _ in range(2):
    cost_action_dict = {}
    opt_ang_list = []
    for action in def_action_list(qubit=qubits):
        def cost(theta):
            return cost_func(qubits, theta, action, action_after_opt, ang_after_opt, rdm , trace_before_diag)
        initial_guess = [0]
        res = minimize(cost, initial_guess, method = 'COBYLA', options={'maxiter': 100})
        cost_action_dict[f'{action}'] = res.fun
        opt_ang_list.append(res.x)
    
    action_after_opt, indx = optimal_action(cost_action_dict)
    ang_after_opt = list(opt_ang_list[indx])

In [ ]:
ang_after_opt